# Importing modules from Jupyter Notebooks

When I started learning to use the Jupyter Notebook,I wanted a way to import modules from other Jupyter Noteooks, I wanted to share what I have learnt and with examples,So I thought why not create a Notebook and share it accross so people can get hands on.

It is a common problem that people want to import code from Jupyter Notebooks. This is made difficult by the fact that Notebooks are not plain Python files, and thus cannot be imported by the regular Python machinery.

Fortunately, Python provides some fairly sophisticated hooks into the import machinery, so we can actually make Jupyter notebooks importable without much difficulty, and only using public APIs.

In [ ]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell



Import hooks typically take the form of two objects:

  1.  A Module Loader, which takes a module name (e.g. 'IPython.display'), and returns a Module
  2.  A Module Finder, which figures out whether a module might exist, and tells Python what Loader to use.
  
In the following example I have written a logic which just accounts for getting path '.ipynb' file
which is basically the module that I want to import.It is adviced for user to write his/her own logic if you want, all we need out of this function is to get the path of '.ipynb' file which has the required module.

In [20]:
def find_notebook(fullname, path=None):
    """
    Example Usage:
    If you are trying to import from a module named add from file add.ipynb,We have to file('.ipynb file')
    Usage:
    find_notebook('add.add')
    """
    name = fullname.rsplit('.', 1)[-1]
    split_path = fullname.rsplit('.', 1)[0]
    split_path = split_path.replace('.','/')
    if not path:
        path = [os.path.abspath('.')+'/'+split_path]
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

## Module Loader

In [ ]:
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("Importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod



Once you know the path of .ipynb file its then a simple thing,you just have to use the module.Example to do that is as bellow.

In [22]:
loader = NotebookLoader()
loaded_notebook = loader.load_module('from_folder.import_me1')
loaded_notebook.import_me1(2,3)

importing Jupyter notebook from /home/shiva/Documents/Python/Jupyter/Notebooks/Importing_from_Notebook/from_folder/import_me1.ipynb


5